# 🚀 Unity Catalog AI: Advanced Developer Workflow Tutorial

## Introduction

Unity Catalog AI 0.2.0 and 0.3.0 introduce groundbreaking enhancements to the developer experience, transforming how data scientists and ML engineers create, test, and deploy functions for AI applications. This tutorial explores these powerful workflow improvements with a focus on scientific computing applications.

Whether you're building AI agents that leverage domain-specific knowledge or creating reusable function libraries for data science workflows, the tools introduced in these releases will dramatically improve your productivity and code quality.

## 🎯 What You'll Learn

This tutorial guides you through a complete developer workflow using real-world astrophysics examples:

- **Function Creation & Registration**: Build sophisticated scientific functions with rich documentation and parameter validation
- **Function Introspection**: Leverage the new ability to retrieve, examine, and modify existing function definitions
- **Execution & Testing**: Validate functions with comprehensive testing across parameter spaces
- **Function Wrapping**: Minimize code duplication by consolidating multiple specialized functions
- **Sandbox Protection**: Ensure system stability with process isolation and resource limits

## 🔍 Why This Matters

The enhancements in Unity Catalog AI 0.2.0 and 0.3.0 address critical pain points in the AI function development lifecycle:

- **Reduced Code Duplication**: Create functions once and reuse them across multiple contexts
- **Improved Safety**: Protect your systems from resource exhaustion and unintended behavior
- **Enhanced Maintainability**: Update existing functions without rewriting them from scratch
- **Better Integration**: Build functions that work seamlessly with modern AI frameworks

Let's explore how these powerful features can transform your workflow for creating AI-powered scientific applications!

## Prerequisites


### Unity Catalog Server

To run this tutorial, you will need:

1. A clone of the [Unity Catalog repository](https://github.com/unitycatalog/unitycatalog).

    ```sh
    git clone https://github.com/unitycatalog/unitycatalog
    ```

2. JDK-17 installed on your system (in order to build and run the Unity Catalog services)<sup>1</sup>
3. [Docker Desktop](https://www.docker.com/products/docker-desktop/) (recommended; you can also install the docker engine yourself)
4. At the repository root, run:

    ```sh
    docker compose up
    ```

<sup>1</sup> (New to managing Java environments? [jenv](https://github.com/jenv/jenv) is a fantastic tool that can help!) 


### Python Requirements

Unity Catalog AI requires Python 3.10+. 

```sh
pip install -U unitycatalog-ai
```

## Setting Up Your Unity Catalog Environment

Before we dive into Unity Catalog AI's powerful features, we need to establish a connection to our Unity Catalog server. 

The 0.3.0 release introduces significant security and execution enhancements. Notice how we're initializing our client with `execution_mode="sandbox"` - this is a new security feature that executes functions in an isolated process pool rather than in the main process. This provides:

- Better resource management with CPU limitations
- Timeout restrictions to prevent runaway processes
- Blocking of modules that might access sensitive system resources

We'll also create a catalog and schema to organize our astrophysics functions, demonstrating the utility methods added in recent releases that simplify catalog and schema management.

In [1]:
from unitycatalog.ai.core.client import UnitycatalogFunctionClient
from unitycatalog.client import ApiClient, Configuration

# Configure the Unity Catalog OSS client
config = Configuration(host="http://localhost:8080/api/2.1/unity-catalog")
api_client = ApiClient(configuration=config)

client = UnitycatalogFunctionClient(api_client=api_client, execution_mode="sandbox")

CATALOG = "astro_catalog"
SCHEMA = "physics_functions"

# Create catalog and schema if they don't already exist
client.uc.create_catalog(name=CATALOG, comment="Catalog for astrophysics calculations")
client.uc.create_schema(
    name=SCHEMA, catalog_name=CATALOG, comment="Schema for astrophysics examples"
)

SchemaInfo(name='physics_functions', catalog_name='astro_catalog', comment='Schema for astrophysics examples', properties={}, full_name='astro_catalog.physics_functions', owner=None, created_at=1742840579002, created_by=None, updated_at=1742840579002, updated_by=None, schema_id='26d02587-86d3-45ed-a8ce-8616cfb02517')

## Crafting Your First Unity Catalog Function

The first step in our development workflow is creating an initial function to register in Unity Catalog. We'll define a function for calculating stellar properties based on astrophysical parameters - a perfect example to showcase how Unity Catalog AI handles complex scientific functions.

This function simulates stellar evolution in a simplified way, calculating key properties like:
- Current mass (after accounting for stellar mass loss over time)
- Luminosity (how bright the star is compared to our Sun)
- Surface temperature (in Kelvin)
- Evolutionary stage (from protostar through main sequence and beyond to end states like white dwarf or black hole)

While this uses simplified models compared to sophisticated stellar evolution codes used in astrophysics research, it demonstrates how domain expertise can be packaged into a callable function that AI assistants can leverage.

Notice the comprehensive docstring in our function definition. This is critical for two reasons:

1. **GenAI Integration**: Unity Catalog AI parses these docstrings to create proper tool definitions when integrating with LLMs like Claude, Gemini, or GPT models. The better your documentation, the more effectively AI agents can understand and use your function.

2. **Developer Experience**: Clear parameter descriptions, return value documentation, and usage examples make your functions more maintainable and easier for other team members to understand.

The 0.2.0 and 0.3.0 releases include enhanced docstring parsing abilities, giving you better control over how your functions appear as tools to AI agents. This function includes parameter validation, meaningful error messages, and clear documentation of parameter constraints - all best practices when creating functions for AI systems.

In [2]:
def calculate_stellar_properties(initial_mass: float, metallicity: float, age: float) -> dict:
    """
    Calculate key stellar characteristics based on initial mass, metallicity, and age.
    This function provides a simplified model of stellar evolution and properties.

    Args:
        initial_mass (float): Initial stellar mass in solar masses (M☉).
            Valid range: 0.1 to 150 solar masses.
        metallicity (float): Stellar metallicity (Z), representing the fraction
            of stellar mass composed of elements heavier than hydrogen and helium.
            Valid range: 0.0001 to 0.04.
        age (float): Stellar age in billions of years.
            Valid range: 0 to 13.8 (age of the universe).

    Returns:
        dict: A dictionary of stellar characteristics including:
            - current_mass: Remaining stellar mass in solar masses
            - luminosity: Current stellar luminosity relative to the Sun
            - surface_temperature: Effective surface temperature in Kelvin
            - stellar_stage: Textual description of the star's evolutionary stage

    Raises:
        ValueError: If input parameters are outside physically meaningful ranges.

    Example:
        >>> result = calculate_stellar_properties(2.5, 0.02, 1.0)
        >>> print(result["luminosity"])
    """
    # Input validation
    if not (0.1 <= initial_mass <= 150):
        raise ValueError("Initial mass must be between 0.1 and 150 solar masses")

    if not (0.0001 <= metallicity <= 0.04):
        raise ValueError("Metallicity must be between 0.0001 and 0.04")

    if not (0 <= age <= 13.8):
        raise ValueError("Stellar age must be between 0 and 13.8 billion years")

    # Calculate stellar lifetime (in billions of years)
    # Massive stars burn fuel much faster - lifetime scales roughly as M^-2.5
    main_sequence_lifetime = 10.0 * (initial_mass / 1.0) ** -2.5

    # Adjust lifetime based on metallicity (metal-poor stars live slightly longer)
    metallicity_factor = (metallicity / 0.02) ** 0.3
    main_sequence_lifetime = main_sequence_lifetime / metallicity_factor

    # Calculate relative age (how far through its lifetime the star is)
    relative_age = min(age / main_sequence_lifetime, 5.0)  # Cap at 5x lifetime for remnants

    # Determine current mass based on evolutionary stage
    if relative_age < 0.1:  # Very early evolution
        # Minimal mass loss during early stages
        current_mass = initial_mass * (1.0 - 0.01 * relative_age)
    elif relative_age < 1.0:  # Main sequence
        # Gradual mass loss during main sequence
        # More massive stars lose more (Wolf-Rayet stars, etc)
        main_sequence_loss = 0.05 * (initial_mass / 10.0) ** 1.5
        current_mass = initial_mass * (1.0 - main_sequence_loss * relative_age)
    elif relative_age < 1.1:  # End of main sequence/subgiant
        # Increased mass loss as star expands
        current_mass = initial_mass * (0.95 - 0.1 * (relative_age - 1.0))
    elif relative_age < 1.3:  # Red giant/AGB phase - dramatic mass loss
        # Up to 50-80% mass loss during giant phases
        giant_phase_factor = 0.5 if initial_mass < 8.0 else 0.3
        current_mass = initial_mass * (0.85 - giant_phase_factor * (relative_age - 1.1) / 0.2)
    else:  # Remnant
        # Determine final remnant mass based on initial mass
        if initial_mass < 8.0:  # White dwarf territory
            # Most stars become 0.5-1.0 M☉ white dwarfs
            current_mass = 0.5 + 0.1 * initial_mass
            current_mass = min(current_mass, 1.4)  # Chandrasekhar limit
        elif initial_mass < 20.0:  # Neutron star territory
            # Stars 8-20 M☉ tend to make ~1.4-2.0 M☉ neutron stars
            current_mass = 1.4 + 0.03 * (initial_mass - 8.0)
        else:  # Black hole territory
            # Stars above 20 M☉ make black holes roughly 10-20% of initial mass
            current_mass = max(3.0, 0.2 * initial_mass)

    # Luminosity calculation
    if relative_age < 1.0:  # Main sequence luminosity
        if current_mass < 0.43:
            luminosity = current_mass**2.3
        elif current_mass < 2:
            luminosity = current_mass**4
        elif current_mass < 20:
            luminosity = current_mass**3.5
        else:
            luminosity = current_mass**3.0
    elif relative_age < 1.3:  # Giant phase luminosity spike
        luminosity = 100 * current_mass**2
    else:  # Remnant luminosity
        if current_mass < 1.4:  # White dwarf
            # White dwarfs cool over time
            cooling_factor = max(0.001, 1.0 - (relative_age - 1.3))
            luminosity = 0.1 * cooling_factor
        elif current_mass < 3.0:  # Neutron star
            luminosity = 0.01  # Very low luminosity
        else:  # Black hole
            # Accretion-dependent, but generally near-zero for isolated black holes
            luminosity = 0.0

    # Surface temperature estimation
    if relative_age < 0.1:  # Protostar
        surface_temperature = 3000 + 1000 * relative_age * 10
    elif relative_age < 1.0:  # Main sequence
        if current_mass < 0.5:
            surface_temperature = 3000 + 1000 * current_mass
        elif current_mass < 1.5:
            surface_temperature = 5000 + 1000 * (current_mass - 0.5)
        else:
            surface_temperature = 6500 + 5000 * (current_mass / 10.0) ** 0.5
    elif relative_age < 1.2:  # Giant phase
        surface_temperature = 4000 - 1000 * (relative_age - 1.0) * 5
    else:  # Remnant
        if current_mass < 1.4:  # White dwarf
            surface_temperature = 20000 * max(0.1, 1.0 - 0.1 * (relative_age - 1.3))
        elif current_mass < 3.0:  # Neutron star
            surface_temperature = 1000000  # Very hot surface
        else:  # Black hole
            surface_temperature = 0  # No surface

    # Stellar stage classification
    if relative_age < 0.01:
        stellar_stage = "Protostar"
    elif relative_age < 1.0:
        stellar_stage = "Main Sequence"
    elif relative_age < 1.05:
        stellar_stage = "Subgiant"
    elif relative_age < 1.2:
        stellar_stage = "Red Giant"
    elif relative_age < 1.3:
        stellar_stage = "Asymptotic Giant Branch"
    else:  # Remnant
        if current_mass < 1.4:
            stellar_stage = "White Dwarf"
        elif current_mass < 3.0:
            stellar_stage = "Neutron Star"
        else:
            stellar_stage = "Black Hole"

    return {
        "current_mass": current_mass,
        "luminosity": luminosity,
        "surface_temperature": surface_temperature,
        "stellar_stage": stellar_stage,
        "main_sequence_lifetime": main_sequence_lifetime,  # In billions of years
    }

## Registering Your First Function with Unity Catalog

Now that we've created our stellar properties function, it's time to register it with Unity Catalog. This step makes our function accessible to AI assistants, Databricks notebooks, and other applications that connect to the Unity Catalog.

The `create_python_function` method handles all the complexity of registering the function. Behind the scenes, it:

1. **Analyzes the function signature** to extract parameter types and return values
2. **Parses the docstring** to extract descriptions, parameter details, and examples
3. **Converts the function** to a format that can be stored and executed by Unity Catalog
4. **Registers the function** in the specified catalog and schema

The `replace=True` parameter allows us to update the function if it already exists, which is particularly useful during development when we're iterating on function implementations.

After registration, we store the fully qualified function name (`{catalog}.{schema}.{function_name}`) for later use. This naming convention is similar to the table naming conventions in databases, providing a hierarchical organization that helps manage complex function libraries.

This registration process creates a clean separation between function development and function execution. You can develop and test functions locally, then register them for broader use once they're ready.

In [3]:
client.create_python_function(
    func=calculate_stellar_properties,
    catalog=CATALOG,
    schema=SCHEMA,
    replace=True,
)

stellar_func_name = f"{CATALOG}.{SCHEMA}.calculate_stellar_properties"

## Registering and Validating Your Function

Now that we've defined our stellar evolution function, let's register it with Unity Catalog and validate it with some test cases. But first, an important disclaimer:

> **Disclaimer**: The stellar evolution model presented here is a dramatic simplification of actual astrophysical processes. Real stellar evolution codes like MESA, STARLIB, or GENEC solve complex sets of differential equations involving nuclear reaction networks, detailed opacity tables, convection modeling, and more. These codes often contain 100,000+ lines of code developed by research teams over decades. Our model captures only the broad patterns of stellar evolution for demonstration purposes.

This section demonstrates one of the most critical aspects of the Unity Catalog AI developer workflow: **testing and validation before deployment**. When working with scientific or analytical functions that will be called by AI agents, it's essential to verify:

1. **Parameter boundaries** work as expected
2. **Edge cases** produce reasonable results
3. **Function outputs** are in expected ranges and formats

The 0.3.0 release makes this validation process much easier with the enhanced sandbox execution mode. As you can see from our test scenarios, we're examining:
- A Sun-like star (1.0 M☉)
- A massive O-type star (25.0 M☉)
- A low-mass red dwarf (0.3 M☉)
- A very massive star that has evolved to a black hole (90.5 M☉)

Verifying these different scenarios helps ensure our function behaves correctly across its parameter space before we expose it to AI systems that might explore edge cases we didn't anticipate.

In [4]:
import ast

scenarios = [
    # Sun-like star
    (1.0, 0.02, 4.6),
    # Massive star
    (25.0, 0.015, 0.002),
    # Low-mass star
    (0.3, 0.001, 10.0),
    # Old Massive Star that has evolved to a singularity
    (90.5, 0.005, 5.0),
]

for mass, metallicity, age in scenarios:
    result = client.execute_function(
        function_name=stellar_func_name,
        parameters={
            "initial_mass": mass,
            "metallicity": metallicity,
            "age": age,
        },
    )
    print(f"\nStellar Model for {mass} M☉ star:")
    for key, value in ast.literal_eval(result.value).items():
        print(f"\t{key}: {value}")


Stellar Model for 1.0 M☉ star:
	current_mass: 0.9992726761381613
	luminosity: 0.9970938770139077
	surface_temperature: 5499.272676138161
	stellar_stage: Main Sequence
	main_sequence_lifetime: 10.0

Stellar Model for 25.0 M☉ star:
	current_mass: 22.16718363676904
	luminosity: 10892.600037747503
	surface_temperature: 13944.323951301596
	stellar_stage: Main Sequence
	main_sequence_lifetime: 0.0034884427442219237

Stellar Model for 0.3 M☉ star:
	current_mass: 0.2999397973798912
	luminosity: 0.06268713412999312
	surface_temperature: 3200.675400362578
	stellar_stage: Main Sequence
	main_sequence_lifetime: 498.3171819730835

Stellar Model for 90.5 M☉ star:
	current_mass: 18.1
	luminosity: 0.0
	surface_temperature: 0
	stellar_stage: Black Hole
	main_sequence_lifetime: 0.00019453446251488867


## Fetching Function Source Code with get_function_source()

One of the most powerful additions in the 0.3.0 release is the ability to retrieve the source code of existing Unity Catalog functions. This capability transforms the development workflow by enabling:

1. **Code Inspection**: Quickly review how existing functions work without having to search through your codebase
2. **Iterative Development**: Make improvements to functions that are already registered
3. **Knowledge Transfer**: Learn from functions written by other team members
4. **Debugging**: Examine function implementations when unexpected results occur

The `get_function_source()` API solves a significant pain point in the Unity Catalog development workflow. Behind the scenes, Unity Catalog stores functions as complex `FunctionInfo` objects that split the function signature, docstring, and body into separate constituent parts within a nested structure. Manually extracting and reconstructing a complete function from this representation is tedious, error-prone, and requires deep knowledge of the internal data model.

With `get_function_source()`, you get back a properly formatted Python function as a string - ready to be modified or used as a template for new functions. This seemingly simple capability eliminates what was previously a frustrating and error-prone manual process of copying metadata fields from complex nested objects.

In the example above, we retrieve the full definition of our stellar properties function. Notice that the entire function, including comments and docstrings, is preserved exactly as it was registered. This allows us to make targeted improvements while maintaining the function's interface and documentation.

In real-world scientific and analytics workflows, functions often evolve as models are refined and improved. The `get_function_source()` API makes this evolution smoother by providing a clear starting point for enhancements.

In [5]:
function_definition = client.get_function_source(function_name=stellar_func_name)

print(function_definition)

def calculate_stellar_properties(initial_mass: float, metallicity: float, age: float) -> dict:
    """
    Calculate key stellar characteristics based on initial mass, metallicity, and age. This function
provides a simplified model of stellar evolution and properties.
    
    Args:
        initial_mass: Initial stellar mass in solar masses (M☉). Valid range: 0.1 to 150 solar masses.
        metallicity: Stellar metallicity (Z), representing the fraction of stellar mass composed of
          elements heavier than hydrogen and helium.  Valid range: 0.0001 to 0.04.
        age: Stellar age in billions of years. Valid range: 0 to 13.8 (age of the universe).
    
    Returns:
        dict
    """
    if not (0.1 <= initial_mass <= 150):
        raise ValueError("Initial mass must be between 0.1 and 150 solar masses")

    if not (0.0001 <= metallicity <= 0.04):
        raise ValueError("Metallicity must be between 0.0001 and 0.04")

    if not (0 <= age <= 13.8):
        raise ValueError("

## Registering the Enhanced Function to Unity Catalog

After developing our enhanced stellar properties function with additional astrophysical parameters, we need to register it to Unity Catalog to make it available for AI assistants and other applications. This step highlights another powerful aspect of the Unity Catalog AI workflow - the ability to maintain multiple versions or variants of related functions within the same catalog.

The registration process for our enhanced function follows the same pattern we used earlier. We call `create_python_function()` with our new enhanced function, using the same catalog and schema but a different function name. Once registered, we store the fully qualified function name for later use.

What's remarkable here is how Unity Catalog AI handles this complex function:

1. **Comprehensive Docstring Analysis**: It parses our detailed documentation, including the expanded return value specification with spectral types, star colors, and other new fields
2. **Parameter Validation Logic**: It preserves all our input validation code, ensuring the function remains safe to call
3. **Complex Calculation Preservation**: It accurately captures all of our sophisticated astrophysics calculations
4. **Local Module Handling**: It properly manages the `math` module that's imported within the function

By storing the qualified function name, we can now call this function just like our original version. This approach allows us to maintain multiple related functions that serve different needs:

- The original function for basic stellar evolution modeling
- The enhanced function for more detailed astrophysical analysis

This pattern is particularly valuable for scientific computing workflows, where you might need both simplified models for quick calculations and more comprehensive models for detailed analysis.

The ability to register multiple versions of related functions also supports an incremental development approach, where you can evolve your functions over time without breaking existing applications that depend on earlier versions.

In [6]:
def calculate_stellar_properties_enhanced(
    initial_mass: float, metallicity: float, age: float
) -> dict:
    """
    Calculate comprehensive stellar characteristics based on initial mass, metallicity, and age.

    This function provides an enhanced model of stellar evolution and properties.

    Args:
        initial_mass (float): Initial stellar mass in solar masses (M☉).
            Valid range: 0.1 to 150 solar masses.
        metallicity (float): Stellar metallicity (Z), representing the fraction
            of stellar mass composed of elements heavier than hydrogen and helium.
            Valid range: 0.0001 to 0.04.
        age (float): Stellar age in billions of years.
            Valid range: 0 to 13.8 (age of the universe).

    Returns:
        dict: A dictionary of stellar characteristics including:
            - current_mass: Remaining stellar mass in solar masses
            - luminosity: Current stellar luminosity relative to the Sun
            - surface_temperature: Effective surface temperature in Kelvin
            - stellar_stage: Textual description of the star's evolutionary stage
            - spectral_type: Harvard spectral classification with luminosity class
            - star_color: Approximate visible color of the star
            - main_sequence_lifetime: Expected lifetime on main sequence in billions of years
            - habitable_zone_inner_radius: Inner boundary of potential planetary habitable zone in AU
            - habitable_zone_outer_radius: Outer boundary of potential planetary habitable zone in AU
            - stellar_wind_velocity: Estimated stellar wind speed in km/s
            - magnetic_field_strength: Estimated magnetic field strength at stellar surface in Gauss

    Raises:
        ValueError: If input parameters are outside physically meaningful ranges.

    Example:
        >>> result = calculate_stellar_properties_enhanced(2.5, 0.02, 1.0)
        >>> print(result["luminosity"])
    """
    # Add local import
    import math

    # Input validation
    if not (0.1 <= initial_mass <= 150):
        raise ValueError("Initial mass must be between 0.1 and 150 solar masses")

    if not (0.0001 <= metallicity <= 0.04):
        raise ValueError("Metallicity must be between 0.0001 and 0.04")

    if not (0 <= age <= 13.8):
        raise ValueError("Stellar age must be between 0 and 13.8 billion years")

    # Calculate stellar lifetime (in billions of years)
    # Massive stars burn fuel much faster - lifetime scales roughly as M^-2.5
    main_sequence_lifetime = 10.0 * (initial_mass / 1.0) ** -2.5

    # Adjust lifetime based on metallicity (metal-poor stars live slightly longer)
    metallicity_factor = (metallicity / 0.02) ** 0.3
    main_sequence_lifetime = main_sequence_lifetime / metallicity_factor

    # Calculate relative age (how far through its lifetime the star is)
    relative_age = min(age / main_sequence_lifetime, 5.0)  # Cap at 5x lifetime for remnants

    # Determine current mass based on evolutionary stage
    if relative_age < 0.1:  # Very early evolution
        # Minimal mass loss during early stages
        current_mass = initial_mass * (1.0 - 0.01 * relative_age)
    elif relative_age < 1.0:  # Main sequence
        # Gradual mass loss during main sequence
        # More massive stars lose more (Wolf-Rayet stars, etc)
        main_sequence_loss = 0.05 * (initial_mass / 10.0) ** 1.5
        current_mass = initial_mass * (1.0 - main_sequence_loss * relative_age)
    elif relative_age < 1.1:  # End of main sequence/subgiant
        # Increased mass loss as star expands
        current_mass = initial_mass * (0.95 - 0.1 * (relative_age - 1.0))
    elif relative_age < 1.3:  # Red giant/AGB phase - dramatic mass loss
        # Up to 50-80% mass loss during giant phases
        giant_phase_factor = 0.5 if initial_mass < 8.0 else 0.3
        current_mass = initial_mass * (0.85 - giant_phase_factor * (relative_age - 1.1) / 0.2)
    else:  # Remnant
        # Determine final remnant mass based on initial mass
        if initial_mass < 8.0:  # White dwarf territory
            # Most stars become 0.5-1.0 M☉ white dwarfs
            current_mass = 0.5 + 0.1 * initial_mass
            current_mass = min(current_mass, 1.4)  # Chandrasekhar limit
        elif initial_mass < 20.0:  # Neutron star territory
            # Stars 8-20 M☉ tend to make ~1.4-2.0 M☉ neutron stars
            current_mass = 1.4 + 0.03 * (initial_mass - 8.0)
        else:  # Black hole territory
            # Stars above 20 M☉ make black holes roughly 10-20% of initial mass
            current_mass = max(3.0, 0.2 * initial_mass)

    # Luminosity calculation
    if relative_age < 1.0:  # Main sequence luminosity
        if current_mass < 0.43:
            luminosity = current_mass**2.3
        elif current_mass < 2:
            luminosity = current_mass**4
        elif current_mass < 20:
            luminosity = current_mass**3.5
        else:
            luminosity = current_mass**3.0
    elif relative_age < 1.3:  # Giant phase luminosity spike
        luminosity = 100 * current_mass**2
    else:  # Remnant luminosity
        if current_mass < 1.4:  # White dwarf
            # White dwarfs cool over time
            cooling_factor = max(0.001, 1.0 - (relative_age - 1.3))
            luminosity = 0.1 * cooling_factor
        elif current_mass < 3.0:  # Neutron star
            luminosity = 0.01  # Very low luminosity
        else:  # Black hole
            # Accretion-dependent, but generally near-zero for isolated black holes
            luminosity = 0.0

    # Surface temperature estimation
    if relative_age < 0.1:  # Protostar
        surface_temperature = 3000 + 1000 * relative_age * 10
    elif relative_age < 1.0:  # Main sequence
        if current_mass < 0.5:
            surface_temperature = 3000 + 1000 * current_mass
        elif current_mass < 1.5:
            surface_temperature = 5000 + 1000 * (current_mass - 0.5)
        else:
            surface_temperature = 6500 + 5000 * (current_mass / 10.0) ** 0.5
    elif relative_age < 1.2:  # Giant phase
        surface_temperature = 4000 - 1000 * (relative_age - 1.0) * 5
    else:  # Remnant
        if current_mass < 1.4:  # White dwarf
            surface_temperature = 20000 * max(0.1, 1.0 - 0.1 * (relative_age - 1.3))
        elif current_mass < 3.0:  # Neutron star
            surface_temperature = 1000000  # Very hot surface
        else:  # Black hole
            surface_temperature = 0  # No surface

    # Stellar stage classification
    if relative_age < 0.01:
        stellar_stage = "Protostar"
    elif relative_age < 1.0:
        stellar_stage = "Main Sequence"
    elif relative_age < 1.05:
        stellar_stage = "Subgiant"
    elif relative_age < 1.2:
        stellar_stage = "Red Giant"
    elif relative_age < 1.3:
        stellar_stage = "Asymptotic Giant Branch"
    else:  # Remnant
        if current_mass < 1.4:
            stellar_stage = "White Dwarf"
        elif current_mass < 3.0:
            stellar_stage = "Neutron Star"
        else:
            stellar_stage = "Black Hole"

    # Enhanced features

    # Habitable zone calculation (only meaningful for main sequence stars)
    # Based on stellar luminosity using simplified model from Kopparapu et al. 2013
    if stellar_stage == "Main Sequence" and surface_temperature > 2600:
        # Convert luminosity to solar units for habitable zone calculation
        # Inner edge (runaway greenhouse limit)
        habitable_zone_inner_radius = 0.75 * (luminosity**0.5)
        # Outer edge (maximum greenhouse limit)
        habitable_zone_outer_radius = 1.77 * (luminosity**0.5)
    else:
        habitable_zone_inner_radius = None
        habitable_zone_outer_radius = None

    # Stellar wind velocity estimation in km/s
    # More sophisticated model based on stellar type and surface temperature
    if stellar_stage in ["Main Sequence", "Subgiant"]:
        if surface_temperature > 30000:  # O stars
            stellar_wind_velocity = 2000 + 500 * (surface_temperature / 40000)
        elif surface_temperature > 10000:  # B and A stars
            stellar_wind_velocity = 1000 + 1000 * (surface_temperature / 30000)
        elif surface_temperature > 5800:  # F and G stars
            stellar_wind_velocity = 450 + 550 * ((surface_temperature - 5800) / 4200)
        else:  # K and M stars
            stellar_wind_velocity = 300 + 150 * ((surface_temperature - 3000) / 2800)
    elif stellar_stage in ["Red Giant", "Asymptotic Giant Branch"]:
        stellar_wind_velocity = 30 + 10 * (luminosity**0.25)  # Slow dense winds
    elif stellar_stage == "White Dwarf":
        stellar_wind_velocity = 4000  # Hot fast winds
    elif stellar_stage == "Neutron Star":
        stellar_wind_velocity = 10000  # Extremely fast winds if active
    else:
        stellar_wind_velocity = 0

    # Magnetic field strength estimation in Gauss
    # Based on empirical relationships for different stellar types
    if stellar_stage == "Main Sequence":
        if surface_temperature > 15000:  # Hot massive stars
            # Hot stars have weaker fields due to fully radiative envelopes
            magnetic_field_strength = 100 * (1 - 0.9 * (age / main_sequence_lifetime))
        elif surface_temperature > 5500:  # Sun-like stars
            # Solar-type stars have moderate fields that decay with age
            magnetic_field_strength = 500 * (1 - 0.7 * (age / main_sequence_lifetime))
        else:  # M dwarfs and cool stars
            # Cool stars can have very strong fields
            rotation_factor = math.exp(-0.5 * age / main_sequence_lifetime)
            magnetic_field_strength = 2000 * rotation_factor
    elif stellar_stage in ["Red Giant", "Asymptotic Giant Branch"]:
        magnetic_field_strength = 10  # Generally weak fields
    elif stellar_stage == "White Dwarf":
        # Some white dwarfs have extremely strong fields
        if current_mass > 1.0:
            magnetic_field_strength = 10**6  # Magnetic white dwarf
        else:
            magnetic_field_strength = 10**3  # Typical white dwarf
    elif stellar_stage == "Neutron Star":
        magnetic_field_strength = 10**12  # Typical neutron star field
    else:
        magnetic_field_strength = 0

    # Chemical enrichment potential (0-100 scale)
    # Indicates how much a star enriches its environment with metals
    if stellar_stage == "Main Sequence":
        if initial_mass > 8:
            chemical_enrichment = 80 + 20 * (
                initial_mass / 40
            )  # Massive stars produce many elements
        else:
            chemical_enrichment = 20 + 10 * initial_mass  # Lower mass stars produce fewer elements
    elif stellar_stage in ["Red Giant", "Asymptotic Giant Branch"]:
        chemical_enrichment = 70 + 30 * min(
            initial_mass / 8, 1.0
        )  # AGB stars produce s-process elements
    elif stellar_stage in ["Neutron Star", "Black Hole"]:
        # Past supernova has enriched environment
        chemical_enrichment = 95 + 5 * min(initial_mass / 30, 1.0)
    else:
        chemical_enrichment = 10  # White dwarfs and other stages contribute little

    # Determine spectral type and color for main sequence stars
    spectral_type = None
    star_color = None

    if stellar_stage == "Main Sequence":
        # Spectral classification based on temperature
        if surface_temperature >= 30000:
            spectral_type = "O"
            star_color = "Blue"
        elif surface_temperature >= 10000:
            spectral_type = "B"
            star_color = "Blue-white"
        elif surface_temperature >= 7500:
            spectral_type = "A"
            star_color = "White"
        elif surface_temperature >= 6000:
            spectral_type = "F"
            star_color = "Yellow-white"
        elif surface_temperature >= 5200:
            spectral_type = "G"
            star_color = "Yellow"
        elif surface_temperature >= 3700:
            spectral_type = "K"
            star_color = "Orange"
        else:
            spectral_type = "M"
            star_color = "Red"

        # Add luminosity class (V for main sequence)
        spectral_type += "V"

    elif stellar_stage == "Subgiant":
        # Subgiants are luminosity class IV
        if surface_temperature >= 10000:
            spectral_type = "B IV"
            star_color = "Blue-white"
        elif surface_temperature >= 7500:
            spectral_type = "A IV"
            star_color = "White"
        elif surface_temperature >= 6000:
            spectral_type = "F IV"
            star_color = "Yellow-white"
        elif surface_temperature >= 5200:
            spectral_type = "G IV"
            star_color = "Yellow"
        elif surface_temperature >= 3700:
            spectral_type = "K IV"
            star_color = "Orange"
        else:
            spectral_type = "M IV"
            star_color = "Red"

    elif stellar_stage in ["Red Giant", "Asymptotic Giant Branch"]:
        # Giants are luminosity class III
        if surface_temperature >= 5200:
            spectral_type = "G III"
            star_color = "Yellow"
        elif surface_temperature >= 3700:
            spectral_type = "K III"
            star_color = "Orange"
        else:
            spectral_type = "M III"
            star_color = "Red"

    elif stellar_stage == "White Dwarf":
        spectral_type = "D"
        star_color = "White"

    elif stellar_stage == "Neutron Star":
        spectral_type = "Neutron Star"
        star_color = None

    elif stellar_stage == "Black Hole":
        spectral_type = "Black Hole"
        star_color = None

    # Create return dictionary with all properties
    result = {
        "current_mass": current_mass,
        "luminosity": luminosity,
        "surface_temperature": surface_temperature,
        "stellar_stage": stellar_stage,
        "spectral_type": spectral_type,
        "star_color": star_color,
        "main_sequence_lifetime": main_sequence_lifetime,
        "habitable_zone_inner_radius": habitable_zone_inner_radius,
        "habitable_zone_outer_radius": habitable_zone_outer_radius,
        "stellar_wind_velocity": stellar_wind_velocity,
        "magnetic_field_strength": magnetic_field_strength,
        "chemical_enrichment_potential": chemical_enrichment,
    }

    return result

In [7]:
client.create_python_function(
    func=calculate_stellar_properties_enhanced,
    catalog=CATALOG,
    schema=SCHEMA,
    replace=True,
)

stellar_func_name_enhanced = f"{CATALOG}.{SCHEMA}.calculate_stellar_properties_enhanced"

## Executing the Enhanced Function with Test Scenarios

After registering our enhanced stellar function with Unity Catalog, we need to validate it with a comprehensive set of test scenarios. This testing phase is critical to ensure our function behaves as expected across the parameter space that AI assistants might explore.

For our enhanced stellar properties function, we've selected four representative scenarios that showcase different stellar types and evolutionary stages:

1. **Sun-like Star**: A G-type main sequence star with solar metallicity and middle age (1.0 M☉)
2. **Massive O-Type Star**: A young, high-mass star that will eventually form a black hole (25.0 M☉)
3. **Low-mass Red Dwarf**: A common, long-lived, cool star (0.3 M☉)
4. **Evolved Massive Star**: An old, massive star that has already formed a black hole (90.5 M☉)

These scenarios test our function across orders of magnitude in stellar mass and through different evolutionary stages, from young main sequence stars to evolved stellar remnants.

For each test case, we:
1. Call the function via `execute_function()` with the appropriate parameters
2. Parse the returned string representation into a Python dictionary
3. Display all the calculated stellar properties for analysis

This systematic testing approach helps catch edge cases and ensures that both fundamental properties (mass, luminosity, temperature) and our enhanced properties (spectral type, habitable zone, magnetic field strength) are calculated correctly for all stellar types.

Testing different evolutionary stages is particularly important for functions that model time-dependent processes. Our test set includes stars in various life stages, from young stars to those that have already completed their evolution, ensuring our function handles the entire stellar lifecycle appropriately.

This validation step provides confidence that when AI assistants or data scientists use our function, it will produce scientifically sound results across the entire parameter space.

In [8]:
scenarios_enhanced = [
    # Sun-like star
    (1.0, 0.02, 4.6),
    # Massive star
    (25.0, 0.015, 0.002),
    # Low-mass star
    (0.3, 0.001, 10.0),
    # Old Massive Star that has evolved to a singularity
    (90.5, 0.005, 5.0),
]

for mass, metallicity, age in scenarios_enhanced:
    result = client.execute_function(
        function_name=stellar_func_name_enhanced,
        parameters={
            "initial_mass": mass,
            "metallicity": metallicity,
            "age": age,
        },
    )
    print(f"\nStellar Model for {mass} M☉ star:")
    for key, value in ast.literal_eval(result.value).items():
        print(f"\t{key}: {value}")


Stellar Model for 1.0 M☉ star:
	current_mass: 0.9992726761381613
	luminosity: 0.9970938770139077
	surface_temperature: 5499.272676138161
	stellar_stage: Main Sequence
	spectral_type: GV
	star_color: Yellow
	main_sequence_lifetime: 10.0
	habitable_zone_inner_radius: 0.748909410957242
	habitable_zone_outer_radius: 1.767426209859091
	stellar_wind_velocity: 433.88960765025865
	magnetic_field_strength: 1589.067205006668
	chemical_enrichment_potential: 30.0

Stellar Model for 25.0 M☉ star:
	current_mass: 22.16718363676904
	luminosity: 10892.600037747503
	surface_temperature: 13944.323951301596
	stellar_stage: Main Sequence
	spectral_type: BV
	star_color: Blue-white
	main_sequence_lifetime: 0.0034884427442219237
	habitable_zone_inner_radius: 78.27571476028163
	habitable_zone_outer_radius: 184.73068683426465
	stellar_wind_velocity: 1464.81079837672
	magnetic_field_strength: 299.3373974219747
	chemical_enrichment_potential: 92.5

Stellar Model for 0.3 M☉ star:
	current_mass: 0.2999397973798912

## Testing a function locally

There can be times when all you want is to retrieve a function definition from Unity Catalog and have it available in your current process, directly as a callable. 

The `get_function_as_callable` API allows the flexibility for several options:

- **Default behavior (non-Jupyter)**: Registers the original function name into the global namespace. You can call the function as if you had defined its implementation in the REPL directly.
- **Global namespace in Jupyter)**: You can directly define the return value of the call to `get_function_as_callable` to the active globals namespace **or** just call the return value as a callable directly (shown in the next cell below)

To set the callable to the global namespace in Jupyter notebooks:

```python
retrieved_func = client.get_function_as_callable(<catalog.schema.func_name>)
globals()["calculate_stellar_properties"] = retrieved_func
```

- **Custom namespace**: If you have a namespace that you would like the callable to be registered in, you can submit a `SimpleNamespace` instance (from the `types` module) to restrict namespace scoping of the loaded function.
- **No registration**: You can set the argument `register_function` to `False`) to only allow for the return value to be used explicitly.

```python
retrieved_func = client.get_function_as_callable(<catalog.schema.func_name>, register_function=False)
# The only way to use this function is by directly calling the return instance.
retrieved_func(initial_mass=1.3, metallicity=0.0002, age=0.65)
```

In [9]:
retrieved_func = client.get_function_as_callable(stellar_func_name)

retrieved_func(initial_mass=99.2, metallicity=0.0005, age=0.005)

{'current_mass': 19.840000000000003,
 'luminosity': 0.0,
 'surface_temperature': 0,
 'stellar_stage': 'Black Hole',
 'main_sequence_lifetime': 0.0003085594253277336}

## Creating Independent Black Hole Physics Functions

To demonstrate the power of function wrapping, let's shift our focus from stellar astrophysics to black hole physics. In this section, we'll create a set of independent functions that calculate various properties of black holes.

Each of these functions represents a distinct physical calculation related to black holes:

1. **Schwarzschild Radius**: Calculates the event horizon radius (the "point of no return") for a black hole of a given mass
2. **Hawking Temperature**: Determines the quantum mechanical thermal radiation emitted by black holes
3. **Event Horizon Area**: Computes the surface area of the event horizon, which relates to black hole entropy
4. **Accretion Luminosity**: Estimates the energy output from matter falling into a black hole
5. **Eddington Luminosity**: Calculates the theoretical maximum luminosity a black hole can achieve

These functions are designed to be independently useful while also serving as building blocks for more comprehensive analyses. Each function:

- Accepts clear, well-defined parameters with validation
- Performs a specific calculation based on established physics equations
- Returns a single, specific physical property
- Includes comprehensive documentation explaining its purpose

This modular approach to function development follows scientific best practices by isolating specific physical phenomena into discrete, testable units. Each function can be validated against known solutions and used independently when only a specific property is needed.

When registering these functions with Unity Catalog, they become individually accessible to AI assistants and other applications. This granularity allows AI systems to select exactly the function they need based on user queries - a question about black hole temperature would invoke just the Hawking temperature function, while a question about accretion would use the relevant accretion function.

This approach also makes the functions more maintainable, as updates to one physical calculation don't affect the others. For example, if a more accurate formula for Hawking radiation is developed, we can update just that function without touching the others.

In [10]:
def schwarzschild_radius(mass_solar_masses: float) -> float:
    """
    Calculate the Schwarzschild radius (event horizon) of a black hole.

    Args:
        mass_solar_masses (float): Mass of the black hole in solar masses.
            Valid range: Greater than 0.

    Returns:
        float: Schwarzschild radius in kilometers.

    Raises:
        ValueError: If mass is not positive.

    Example:
        >>> schwarzschild_radius(10)
        29.5327
    """
    if mass_solar_masses <= 0:
        raise ValueError("Mass must be positive")

    # Convert solar masses to kg
    mass_kg = mass_solar_masses * 1.989e30

    # Constants
    G = 6.67430e-11  # Gravitational constant in m^3 kg^-1 s^-2
    c = 299792458  # Speed of light in m/s

    # Calculate Schwarzschild radius in meters
    r_s_meters = 2 * G * mass_kg / c**2

    # Convert to kilometers
    r_s_km = r_s_meters / 1000

    return r_s_km


def hawking_temperature(mass_solar_masses: float) -> float:
    """
    Calculate the Hawking radiation temperature of a black hole.

    Args:
        mass_solar_masses (float): Mass of the black hole in solar masses.
            Valid range: Greater than 0.

    Returns:
        float: Hawking radiation temperature in nanokelvin.

    Raises:
        ValueError: If mass is not positive.

    Example:
        >>> hawking_temperature(10)
        6.1707e-10
    """
    import math

    if mass_solar_masses <= 0:
        raise ValueError("Mass must be positive")

    # Convert solar masses to kg
    mass_kg = mass_solar_masses * 1.989e30

    # Constants
    h_bar = 1.054571817e-34  # Reduced Planck constant in J·s
    c = 299792458  # Speed of light in m/s
    G = 6.67430e-11  # Gravitational constant in m^3 kg^-1 s^-2
    k_B = 1.380649e-23  # Boltzmann constant in J/K

    # Calculate Hawking temperature in Kelvin
    T = (h_bar * c**3) / (8 * math.pi * G * k_B * mass_kg)

    # Convert to nanokelvin for more readable numbers
    T_nK = T * 1e9

    return T_nK


def event_horizon_area(mass_solar_masses: float) -> float:
    """
    Calculate the surface area of a black hole's event horizon.

    Args:
        mass_solar_masses (float): Mass of the black hole in solar masses.
            Valid range: Greater than 0.

    Returns:
        float: Event horizon surface area in square kilometers.

    Raises:
        ValueError: If mass is not positive.

    Example:
        >>> event_horizon_area(10)
        10955.5
    """
    import math

    if mass_solar_masses <= 0:
        raise ValueError("Mass must be positive")

    # Constants
    G = 6.67430e-11  # Gravitational constant in m^3 kg^-1 s^-2
    c = 299792458  # Speed of light in m/s

    # Convert solar masses to kg
    mass_kg = mass_solar_masses * 1.989e30

    # Calculate Schwarzschild radius in meters
    r_s_meters = 2 * G * mass_kg / c**2

    # Convert to kilometers
    r_s_km = r_s_meters / 1000

    # Calculate area in square kilometers
    area = 4 * math.pi * r_s_km**2

    return area


def accretion_luminosity(
    mass_solar_masses: float, accretion_rate_solar_masses_per_year: float, efficiency: float = 0.1
) -> float:
    """
    Calculate the luminosity from matter accreting onto a black hole.

    Args:
        mass_solar_masses (float): Mass of the black hole in solar masses.
            Valid range: Greater than 0.
        accretion_rate_solar_masses_per_year (float): Rate at which matter falls
            into the black hole in solar masses per year.
            Valid range: Greater than or equal to 0.
        efficiency (float, optional): Efficiency of converting rest mass to radiation.
            Default is 0.1 (10%). Valid range: 0 to 1.

    Returns:
        float: Luminosity in units of solar luminosity.

    Raises:
        ValueError: If parameters are outside valid ranges.

    Example:
        >>> accretion_luminosity(10, 1e-9)
        5.7e4
    """
    if mass_solar_masses <= 0:
        raise ValueError("Mass must be positive")
    if accretion_rate_solar_masses_per_year < 0:
        raise ValueError("Accretion rate cannot be negative")
    if not 0 <= efficiency <= 1:
        raise ValueError("Efficiency must be between 0 and 1")

    # Constants
    c = 299792458  # Speed of light in m/s
    L_sun = 3.828e26  # Solar luminosity in watts
    M_sun = 1.989e30  # Solar mass in kg

    # Convert solar masses per year to kg/s
    accretion_rate_kg_s = accretion_rate_solar_masses_per_year * M_sun / (365.25 * 24 * 3600)

    # Calculate luminosity in watts
    L_watts = efficiency * accretion_rate_kg_s * c**2

    # Convert to solar luminosity
    L_solar = L_watts / L_sun

    return L_solar


def eddington_luminosity(mass_solar_masses: float) -> float:
    """
    Calculate the Eddington luminosity of a black hole.

    The Eddington luminosity is the maximum luminosity a black hole can achieve
    through accretion while maintaining hydrostatic equilibrium, where radiation
    pressure balances gravitational attraction.

    Args:
        mass_solar_masses (float): Mass of the black hole in solar masses.
            Valid range: Greater than 0.

    Returns:
        float: Eddington luminosity in units of solar luminosity.

    Raises:
        ValueError: If mass is not positive.

    Example:
        >>> eddington_luminosity(10)
        3.2e38
    """
    import math

    if mass_solar_masses <= 0:
        raise ValueError("Mass must be positive")

    # Constants
    G = 6.67430e-11  # Gravitational constant in m^3 kg^-1 s^-2
    c = 299792458  # Speed of light in m/s
    thomson_cross_section = 6.65e-29  # Thomson cross-section for electron in m²
    m_p = 1.67262192e-27  # Proton mass in kg
    L_sun = 3.828e26  # Solar luminosity in watts

    # Convert solar masses to kg
    mass_kg = mass_solar_masses * 1.989e30

    # Calculate Eddington luminosity in watts
    L_edd_watts = 4 * math.pi * G * mass_kg * m_p * c / thomson_cross_section

    # Convert to solar luminosity
    L_edd_solar = L_edd_watts / L_sun

    return L_edd_solar

## Testing Individual Functions Before Wrapping

Before consolidating our functions into a wrapper, it's crucial to register and test each component function independently. This step ensures that each calculation works correctly on its own and helps isolate any issues before they propagate to more complex functions.

Unity Catalog AI makes this process straightforward with the `create_python_function` API, which allows us to register each function with just a single call:

```python
for func in [schwarzschild_radius, hawking_temperature, event_horizon_area, 
             accretion_luminosity, eddington_luminosity]:
    client.create_python_function(catalog=CATALOG, schema=SCHEMA, func=func, replace=True)
```

This approach has several key benefits for the development workflow:

1. **Incremental Testing**: You can validate each specialized calculation separately
2. **Independent Accessibility**: AI agents and other systems can call individual functions when they only need specific calculations
3. **Clearer Debugging**: Issues can be pinpointed to specific functions rather than troubleshooting the entire wrapper
4. **Versioning Flexibility**: You can update individual functions without necessarily having to update the wrapper

For scientific applications like our black hole physics library, this modular approach mirrors how scientific understanding develops - specific phenomena are studied individually before being integrated into comprehensive models.

Testing these functions individually also provides confidence that the wrapper function will work correctly, as it's built on validated components. This incremental validation approach is a best practice for complex scientific and analytical workflows.

In [11]:
for func in [
    schwarzschild_radius,
    hawking_temperature,
    event_horizon_area,
    accretion_luminosity,
    eddington_luminosity,
]:
    client.create_python_function(catalog=CATALOG, schema=SCHEMA, func=func, replace=True)

In [12]:
client.execute_function(
    function_name=f"{CATALOG}.{SCHEMA}.schwarzschild_radius",
    parameters={"mass_solar_masses": 1.9934234e47},
)

FunctionExecutionResult(error=None, format='SCALAR', value=5.8888250014088315e+47, truncated=None)

In [13]:
client.execute_function(
    function_name=f"{CATALOG}.{SCHEMA}.hawking_temperature",
    parameters={"mass_solar_masses": 1.9934234e47},
)

FunctionExecutionResult(error=None, format='SCALAR', value=3.0943901394108388e-46, truncated=None)

In [14]:
client.execute_function(
    function_name=f"{CATALOG}.{SCHEMA}.event_horizon_area",
    parameters={"mass_solar_masses": 1.912e147},
)

FunctionExecutionResult(error=None, format='SCALAR', value=4.009071789077358e+296, truncated=None)

In [15]:
client.execute_function(
    function_name=f"{CATALOG}.{SCHEMA}.accretion_luminosity",
    parameters={
        "mass_solar_masses": 1.912e147,
        "accretion_rate_solar_masses_per_year": 1.1,
        "efficiency": 0.8,
    },
)

FunctionExecutionResult(error=None, format='SCALAR', value=13022158135851.205, truncated=None)

In [16]:
client.execute_function(
    function_name=f"{CATALOG}.{SCHEMA}.eddington_luminosity",
    parameters={"mass_solar_masses": 4.998},
)

FunctionExecutionResult(error=None, format='SCALAR', value=164237.2010417032, truncated=None)

## Function Wrapping: Creating Integrated Solutions with Minimal Code Duplication

One of the most powerful features introduced in Unity Catalog AI 0.3.0 is the `create_wrapped_function` API. This capability solves a fundamental challenge in modular programming: how to combine specialized functions into more powerful composite functions without duplicating code. Let's explore how this works in the context of our black hole physics library.

### The Problem: Function Dependencies in Unity Catalog

Unity Catalog functions are registered independently and can't directly call each other within the catalog environment. This creates a dilemma when you need functions to build on each other's results:

1. You could **duplicate logic** across functions, but this leads to maintenance nightmares
2. You could make **client-side calls** between functions, but this adds latency and complexity
3. You could maintain **separate codebases** for individual functions and composites, but this creates version drift

### The Solution: Automatic Function Wrapping

The `create_wrapped_function` API elegantly solves this problem by:

1. **Automatically injecting** the source code of dependent functions
2. **Preserving direct function calls** in your wrapper logic
3. **Creating a single consolidated function** in Unity Catalog that contains all necessary code

With this approach, you can:
- Develop and test each component function independently
- Combine them into powerful composite solutions
- Maintain all functions from a single codebase
- Avoid code duplication and synchronization challenges

In our black hole physics example, the `black_hole_analyzer` wrapper directly calls the five specialized functions (`schwarzschild_radius`, `event_horizon_area`, etc.) as if they were locally available. When registered with `create_wrapped_function`, the system automatically injects all necessary function code to make a self-contained composite function.

This pattern is particularly valuable for scientific and analytical workflows, where complex analyses often build upon layers of specialized calculations. The wrapper can focus on integration logic and derived calculations while leveraging the specialized expertise in each component function.

In [17]:
# Define the wrapper function
def black_hole_analyzer(
    mass_solar_masses: float,
    accretion_rate_solar_masses_per_year: float = 0.0,
    efficiency: float = 0.1,
) -> dict:
    """
    Comprehensive analysis of black hole properties based on mass and accretion.

    This function acts as a wrapper that calls multiple black hole physics
    functions and combines their results into a single comprehensive report.
    It also performs additional calculations based on the results of these functions.

    Args:
        mass_solar_masses: Mass of the black hole in solar masses.
            Valid range: Greater than 0.
        accretion_rate_solar_masses_per_year: Rate at which matter falls
            into the black hole in solar masses per year. Default is 0.0 (no accretion).
        efficiency: Efficiency of converting rest mass to radiation
            during accretion. Default is 0.1 (10%). Valid range: 0 to 1.

    Returns:
        dict: A dictionary containing multiple black hole properties:
            - event_horizon_radius_km: Schwarzschild radius in kilometers
            - event_horizon_area_km2: Surface area of the event horizon in square kilometers
            - hawking_temperature_nK: Hawking radiation temperature in nanokelvin
            - accretion_luminosity_solar: Luminosity from accretion in units of solar luminosity
            - eddington_luminosity_solar: Maximum sustainable luminosity in solar units
            - eddington_ratio: Ratio of accretion luminosity to Eddington luminosity
            - time_to_evaporation_years: Estimated time until evaporation due to Hawking radiation
            - photon_sphere_radius_km: Radius of the photon sphere where light orbits the black hole
            - innermost_stable_orbit_km: Radius of the innermost stable circular orbit

    Example:
        >>> result = black_hole_analyzer(10, 1e-9)
        >>> print(f"Event horizon: {result['event_horizon_radius_km']:.2f} km")
        Event horizon: 29.53 km
    """
    # The function bodies will be automatically injected by create_wrapped_function
    # so we can directly call the functions that will be wrapped

    # Get results from the independent functions
    event_horizon_radius_km = schwarzschild_radius(mass_solar_masses)
    event_horizon_area_km2 = event_horizon_area(mass_solar_masses)
    hawking_temperature_nK = hawking_temperature(mass_solar_masses)
    accretion_luminosity_solar = accretion_luminosity(
        mass_solar_masses, accretion_rate_solar_masses_per_year, efficiency
    )
    eddington_luminosity_solar = eddington_luminosity(mass_solar_masses)

    # Calculate additional derived properties

    # Eddington ratio (L/L_edd)
    eddington_ratio = (
        accretion_luminosity_solar / eddington_luminosity_solar
        if eddington_luminosity_solar > 0
        else 0
    )

    # Calculate photon sphere radius (where photons orbit the black hole)
    photon_sphere_radius_km = 1.5 * event_horizon_radius_km

    # Calculate innermost stable circular orbit (ISCO)
    innermost_stable_orbit_km = 3.0 * event_horizon_radius_km

    # Estimate black hole evaporation time due to Hawking radiation
    # Constants
    import math

    G = 6.67430e-11  # Gravitational constant in m^3 kg^-1 s^-2
    c = 299792458  # Speed of light in m/s
    hbar = 1.054571817e-34  # Reduced Planck constant in J·s
    M_sun = 1.989e30  # Solar mass in kg

    # Calculation (approximation based on Stephen Hawking's work)
    # t_evap ≈ 5120 * π * G^2 * M^3 / (hbar * c^4)
    mass_kg = mass_solar_masses * M_sun
    evaporation_time_seconds = 5120 * math.pi * G**2 * mass_kg**3 / (hbar * c**4)
    seconds_per_year = 365.25 * 24 * 3600
    time_to_evaporation_years = evaporation_time_seconds / seconds_per_year

    # Return the comprehensive analysis
    return {
        "event_horizon_radius_km": event_horizon_radius_km,
        "event_horizon_area_km2": event_horizon_area_km2,
        "hawking_temperature_nK": hawking_temperature_nK,
        "accretion_luminosity_solar": accretion_luminosity_solar,
        "eddington_luminosity_solar": eddington_luminosity_solar,
        "eddington_ratio": eddington_ratio,
        "time_to_evaporation_years": time_to_evaporation_years,
        "photon_sphere_radius_km": photon_sphere_radius_km,
        "innermost_stable_orbit_km": innermost_stable_orbit_km,
    }


# Register the wrapped function
wrapped_function = client.create_wrapped_function(
    primary_func=black_hole_analyzer,
    functions=[
        schwarzschild_radius,
        event_horizon_area,
        hawking_temperature,
        accretion_luminosity,
        eddington_luminosity,
    ],
    catalog=CATALOG,
    schema=SCHEMA,
    replace=True,
)

## Executing Wrapped Functions: Testing the Final Product

The final stage in our Unity Catalog AI development workflow is executing and testing our wrapped function. This section demonstrates how to call our consolidated `black_hole_analyzer` function and interpret its comprehensive results.

We'll analyze three different classes of black holes to showcase the power of our wrapped function:

1. **Stellar Mass Black Hole (10 M☉)** - Similar to famous black holes like Cygnus X-1, these form from the collapse of massive stars
2. **Intermediate Mass Black Hole (1,000 M☉)** - A rarer class that might form in dense star clusters
3. **Supermassive Black Hole (4 million M☉)** - Like Sagittarius A* at the center of our Milky Way galaxy

The execution is straightforward - we call the function with the appropriate parameters and receive a comprehensive analysis of each black hole's properties. Note how the wrapper function automatically leverages all the individual component functions while adding additional derived calculations.

This example illustrates why the Unity Catalog AI 0.3.0 enhancements are so powerful for scientific computing workflows:

1. **Modularity**: Each specialized calculation is maintained in its own function
2. **Consolidation**: The wrapper brings everything together without code duplication
3. **Extensibility**: New calculations can be added to either component functions or the wrapper
4. **Clarity**: Complex physics is organized into logical units with clear documentation

For data scientists and researchers, this pattern dramatically improves development efficiency while maintaining the specialized nature of scientific calculations. It's particularly valuable when creating functions for AI assistants, as it allows you to present complex scientific calculations through well-documented, logically structured interfaces.

In [18]:
# Execute our wrapped black hole analyzer function with a variety of black hole masses
import ast
from pprint import pprint

# Stellar mass black hole (10 solar masses, like Cygnus X-1)
stellar_bh_result = client.execute_function(
    function_name=f"{CATALOG}.{SCHEMA}.black_hole_analyzer",
    parameters={
        "mass_solar_masses": 10.0,
        "accretion_rate_solar_masses_per_year": 1e-8,
        "efficiency": 0.1,
    },
)
print("Stellar Mass Black Hole (10 M☉):")
pprint(ast.literal_eval(stellar_bh_result.value))
print("\n" + "-" * 80 + "\n")

# Intermediate mass black hole (1,000 solar masses)
intermediate_bh_result = client.execute_function(
    function_name=f"{CATALOG}.{SCHEMA}.black_hole_analyzer",
    parameters={"mass_solar_masses": 1000.0, "accretion_rate_solar_masses_per_year": 1e-6},
)
print("Intermediate Mass Black Hole (1,000 M☉):")
pprint(ast.literal_eval(intermediate_bh_result.value))
print("\n" + "-" * 80 + "\n")

# Supermassive black hole (4 million solar masses, like Sagittarius A*)
smbh_result = client.execute_function(
    function_name=f"{CATALOG}.{SCHEMA}.black_hole_analyzer", parameters={"mass_solar_masses": 4.0e6}
)
print("Supermassive Black Hole (4 million M☉, like Sgr A*):")
pprint(ast.literal_eval(smbh_result.value))

Stellar Mass Black Hole (10 M☉):
{'accretion_luminosity_solar': 14797.906972558185,
 'eddington_luminosity_solar': 328605.84442117484,
 'eddington_ratio': 0.045032391309485276,
 'event_horizon_area_km2': 10966.500359645965,
 'event_horizon_radius_km': 29.54126555055405,
 'hawking_temperature_nK': 6.168429712630828,
 'innermost_stable_orbit_km': 88.62379665166216,
 'photon_sphere_radius_km': 44.31189832583108,
 'time_to_evaporation_years': 2.0973585980140657e+70}

--------------------------------------------------------------------------------

Intermediate Mass Black Hole (1,000 M☉):
{'accretion_luminosity_solar': 1479790.6972558186,
 'eddington_luminosity_solar': 32860584.442117486,
 'eddington_ratio': 0.045032391309485276,
 'event_horizon_area_km2': 109665003.59645963,
 'event_horizon_radius_km': 2954.126555055405,
 'hawking_temperature_nK': 0.061684297126308275,
 'innermost_stable_orbit_km': 8862.379665166214,
 'photon_sphere_radius_km': 4431.189832583107,
 'time_to_evaporation_year

## Sandbox Protection: Safeguarding Against Dangerous Executions

One of the most powerful security enhancements in Unity Catalog AI 0.3.0 is the improved sandbox execution mode. This feature provides critical protection for production systems by isolating potentially hazardous function executions in separate processes.

### Why Sandbox Protection Matters

When working with AI agents that can call arbitrary functions, you face a fundamental challenge: users might unknowingly (or deliberately) provide parameters that could:

1. **Crash your system** with unbounded computational demands
2. **Lock up resources** with infinite loops or excessive memory usage
3. **Access sensitive system resources** through imported modules
4. **Block other operations** by monopolizing computational resources

The sandbox execution mode creates a protective barrier by running functions in isolated processes with strict resource limitations.

### How Sandbox Protection Works

In this section, we'll demonstrate how the sandbox works by creating a deliberately resource-intensive function: a Mandelbrot set calculator. While this function is perfectly safe with reasonable parameters, it can consume enormous computational resources when given large dimensions or iteration counts.

The sandbox protection system:

1. **Runs code in isolated processes** rather than in the main application process
2. **Enforces CPU time limits** to prevent excessive computation
3. **Restricts memory usage** to prevent memory exhaustion
4. **Blocks access to sensitive system modules** like `os`, `sys`, and `subprocess`
5. **Imposes execution timeouts** to ensure functions terminate

For our demonstration, we've customized the timeout to just 3 seconds (compared to the default 30 seconds) to show how quickly the system can respond to potentially problematic executions.

This protection is especially important for GenAI applications, where large language models might explore edge cases or extreme parameters that human users would typically avoid. By configuring the sandbox mode appropriately, you can ensure that even the most computationally demanding function calls will fail gracefully rather than bringing down your entire application.

## ⚠️ Computational Complexity Warning: Mandelbrot Set Calculator

The `mandelbrot_set_calculator` function provides a fascinating mathematical tool for generating the famous Mandelbrot set fractal. However, it presents significant computational challenges that make it a perfect example of why unrestricted function execution in AI agents can be dangerous.

### The Computational Complexity Problem

#### Time Complexity
- **O(width × height × max_iterations)** - This is a cubic growth pattern in the worst case
- With maximum allowed parameters (5000×5000 pixels, 100,000 iterations), this results in **2.5 trillion operations**
- A single call with large parameters can monopolize CPU resources for minutes or even hours

#### Memory Consumption
- Creates a NumPy array of size `width × height × 4 bytes` (for int32)
- At maximum dimensions (5000×5000), requires **~95MB** just for the result matrix
- Additional memory overhead for the Python lists created for JSON serialization

### Dangers in GenAI Agent Contexts

If an AI agent has unrestricted access to execute this function:

1. **Denial of Service (DoS)**: A user could easily (intentionally or unintentionally) trigger a computation that paralyzes the main application process

2. **Resource Starvation**: A single request with parameters like `width=5000, height=5000, max_iterations=100000` could consume all available CPU resources

3. **Unpredictable Response Times**: Even with moderate parameters, execution time varies dramatically based on the specific region of the complex plane being calculated

4. **Memory Exhaustion**: Large result matrices can consume significant memory, potentially causing OOM (Out of Memory) errors

## Safeguard Solutions

This is why process/queue-based sandbox execution with the following protections is essential:

- **CPU Limits**: Restrict the maximum CPU time any single calculation can consume
- **Memory Caps**: Prevent memory usage from exceeding predefined thresholds
- **Execution Timeouts**: Automatically terminate calculations that exceed time limits
- **Process Isolation**: Run computationally intensive functions in separate processes to protect the main application
- **Queue Management**: Prevent queue flooding by limiting concurrent intensive operations

### Practical Example

Consider these two parameter sets:

1. `width=100, height=100, max_iterations=1000`
   - ~10 million operations
   - Completes almost instantly
   - Memory usage: ~40KB

2. `width=5000, height=5000, max_iterations=50000`
   - ~1.25 trillion operations
   - Could take hours on standard hardware
   - Memory usage: ~95MB

Without proper safeguards, the second example could effectively crash or freeze your application.

In [19]:
def mandelbrot_set_calculator(
    width: int,
    height: int,
    max_iterations: int = 1000,
    x_min: float = -2.0,
    x_max: float = 1.0,
    y_min: float = -1.5,
    y_max: float = 1.5,
) -> dict:
    """
    Calculates the Mandelbrot set for the specified parameters.

    This function creates a 2D array representing the Mandelbrot set. Each element
    in the array contains the number of iterations required to determine if the
    corresponding point in the complex plane is in the Mandelbrot set.

    WARNING: This function can be computationally expensive with large dimensions or
    high iteration counts. Values above 2000x2000 pixels or 10000 iterations may
    cause excessive CPU usage or memory consumption.

    Args:
        width (int): Width of the resulting image in pixels.
            Valid range: 1 to 5000.
        height (int): Height of the resulting image in pixels.
            Valid range: 1 to 5000.
        max_iterations (int, optional): Maximum number of iterations to determine if a point
            is in the set. Defaults to 1000.
            Valid range: 10 to 100000.
        x_min (float, optional): Minimum real value of the complex plane. Defaults to -2.0.
        x_max (float, optional): Maximum real value of the complex plane. Defaults to 1.0.
        y_min (float, optional): Minimum imaginary value of the complex plane. Defaults to -1.5.
        y_max (float, optional): Maximum imaginary value of the complex plane. Defaults to 1.5.

    Returns:
        dict: A dictionary containing:
            - dimensions: Tuple of (width, height)
            - iterations_matrix: 2D list of iteration counts
            - calculation_info: Information about the calculation parameters

    Raises:
        ValueError: If width, height, or max_iterations are outside their valid ranges.

    Example:
        >>> result = mandelbrot_set_calculator(100, 100, 500)
        >>> print(
        ...     f"Generated a Mandelbrot set of {result['dimensions'][0]}x{result['dimensions'][1]} pixels"
        ... )
    """
    # Input validation
    if not (1 <= width <= 5000):
        raise ValueError("Width must be between 1 and 5000 pixels")
    if not (1 <= height <= 5000):
        raise ValueError("Height must be between 1 and 5000 pixels")
    if not (10 <= max_iterations <= 100000):
        raise ValueError("Max iterations must be between 10 and 100000")

    # Import NumPy here to keep it isolated to this function
    import numpy as np

    # Create a 2D array to store the iteration counts
    result_matrix = np.zeros((height, width), dtype=np.int32)

    # Calculate the step size in the complex plane
    x_step = (x_max - x_min) / width
    y_step = (y_max - y_min) / height

    # Loop through each pixel
    for y in range(height):
        for x in range(width):
            # Convert pixel coordinate to complex number
            c_real = x_min + x * x_step
            c_imag = y_min + y * y_step
            c = complex(c_real, c_imag)

            # Perform Mandelbrot iteration
            z = complex(0, 0)
            iteration = 0

            while abs(z) <= 2 and iteration < max_iterations:
                z = z**2 + c
                iteration += 1

            # Store the iteration count
            result_matrix[y, x] = iteration

    # Convert the NumPy array to a Python list for JSON serialization
    iterations_list = result_matrix.tolist()

    # Create the return dictionary
    result = {
        "dimensions": (width, height),
        "iterations_matrix": iterations_list,
        "calculation_info": {
            "max_iterations": max_iterations,
            "complex_plane_bounds": {
                "x_min": x_min,
                "x_max": x_max,
                "y_min": y_min,
                "y_max": y_max,
            },
        },
    }

    return result

## Sandbox Protection: Safeguarding Against Runaway Computations

One of the most significant security enhancements in Unity Catalog AI 0.3.0 is the improved sandbox execution mode. This feature provides critical protection for production systems when integrating with AI agents or handling user-provided parameters.

### The Problem: Dangerous Function Executions

When exposing functions to AI agents or end users, several risks emerge:

1. **Resource Exhaustion**: Functions with unbounded computation can consume all available CPU or memory
2. **Denial of Service**: Poorly parameterized functions can effectively lock up your system
3. **System Access**: Malicious code might attempt to access sensitive system resources
4. **Infinite Loops**: Functions without proper termination conditions can run indefinitely

These issues are particularly concerning with GenAI applications because large language models may explore edge cases or suggest extreme parameters that could unintentionally stress your system.

### The Solution: Process Isolation with Resource Limits

The sandbox execution mode in Unity Catalog AI 0.3.0 provides comprehensive protection through several mechanisms:

1. **Process Isolation**: Functions run in separate processes rather than in your main application
2. **Resource Limitations**:
  - **CPU Time Limits**: Restricts total CPU time consumption
  - **Memory Caps**: Prevents memory exhaustion
  - **Execution Timeouts**: Forces termination after a specified wall clock time
3. **Module Restrictions**: Blocks access to dangerous system modules like `os`, `sys`, and `subprocess`
4. **File Access Prevention**: Disables the built-in `open()` function to prevent filesystem access

### Configuration Through Environment Variables

The sandbox protection system is highly configurable through environment variables:

- `EXECUTOR_TIMEOUT`: Maximum wall clock time (in seconds) before termination
- `EXECUTOR_MAX_CPU_TIME_LIMIT`: Maximum CPU time (in seconds)
- `EXECUTOR_MAX_MEMORY_LIMIT`: Maximum memory usage (in MB)
- `EXECUTOR_DISALLOWED_MODULES`: Comma-separated list of modules to block

As demonstrated in our example, you can adjust these settings to match your specific requirements and risk tolerance.

### The Mandelbrot Example

Our Mandelbrot set calculator provides a perfect demonstration of how sandbox protection works:

- With modest parameters (50x50 pixels), the function completes quickly and returns valid results
- With extreme parameters (3000x3000 pixels), the computation would normally consume significant resources and time
- The sandbox detects the excessive computation and terminates it after our configured 3-second timeout

This protection mechanism ensures that even if an AI agent or user submits problematic parameters, your system remains responsive and stable. The function simply fails gracefully with a timeout message rather than bringing down your entire application.

For production GenAI applications, this layer of protection is essential when exposing computational functions to large language models, which might not inherently understand the resource implications of the parameters they suggest.

In [20]:
# Register the Mandelbrot function
client.create_python_function(
    catalog=CATALOG, schema=SCHEMA, func=mandelbrot_set_calculator, replace=True
)

print(f"Registered function: {CATALOG}.{SCHEMA}.mandelbrot_set_calculator")

# Import the required modules for environment configuration
import os

# Save the original timeout value if it exists
original_timeout = os.environ.get("EXECUTOR_TIMEOUT", "30")
print(f"Original execution timeout: {original_timeout} seconds")

# Modify the timeout to a lower value for demonstration
# This would terminate computations after just 3 seconds
os.environ["EXECUTOR_TIMEOUT"] = "3"
print(f"Modified execution timeout: {os.environ['EXECUTOR_TIMEOUT']} seconds")

try:
    print("\n--- Safe Parameters Test ---")
    # Execute with safe parameters
    safe_result = client.execute_function(
        function_name=f"{CATALOG}.{SCHEMA}.mandelbrot_set_calculator",
        parameters={"width": 50, "height": 50, "max_iterations": 100},
    )
    import ast

    safe_result_dict = ast.literal_eval(safe_result.value)
    print(
        f"Safe execution successful! Generated a {safe_result_dict['dimensions'][0]}x{safe_result_dict['dimensions'][1]} matrix"
    )

    print("\n--- Dangerous Parameters Test ---")
    # Execute with parameters that would timeout
    print("Attempting execution with parameters that will exceed the timeout...")
    dangerous_result = client.execute_function(
        function_name=f"{CATALOG}.{SCHEMA}.mandelbrot_set_calculator",
        parameters={"width": 3000, "height": 3000, "max_iterations": 5000},
    )
    print(f"Result: {dangerous_result.error}")

except Exception as e:
    print(f"An error occurred: {str(e)}")
finally:
    # Restore the original timeout
    os.environ["EXECUTOR_TIMEOUT"] = original_timeout
    print(
        f"\nRestored execution timeout to original value: {os.environ['EXECUTOR_TIMEOUT']} seconds"
    )

Registered function: astro_catalog.physics_functions.mandelbrot_set_calculator
Original execution timeout: 30 seconds
Modified execution timeout: 3 seconds

--- Safe Parameters Test ---
Safe execution successful! Generated a 50x50 matrix

--- Dangerous Parameters Test ---
Attempting execution with parameters that will exceed the timeout...
Result: The function execution has timed out and has been canceled due to excessive resource consumption.
There are two timeout conditions to consider:
	1. You can increase the CPU execution timeout by setting the environment variable EXECUTOR_MAX_CPU_TIME_LIMIT. The default value is 10 seconds.
	2. You can increase the wall-clock time limit by setting the environment variable EXECUTOR_TIMEOUT. The default value is 20 seconds.


Restored execution timeout to original value: 30 seconds


## Conclusion: Building Robust Scientific Functions with Unity Catalog AI

In this tutorial, we've explored the powerful development workflow enhancements introduced in Unity Catalog AI versions 0.2.0 and 0.3.0. These features significantly improve the development experience when creating, testing, and deploying scientific functions for AI-powered applications.

### Key Workflow Improvements We've Covered

1. **Enhanced Function Management**
  - Setting up connections to Unity Catalog
  - Creating and registering complex scientific functions
  - Retrieving function source code for inspection and modification
  - Updating functions with new capabilities

2. **Function Consolidation**
  - Creating independent scientific calculation functions
  - Building wrapper functions that consolidate multiple calculations
  - Minimizing code duplication through function wrapping
  - Providing comprehensive interfaces for AI agents

3. **Safety and Security**
  - Executing functions in sandbox mode for process isolation
  - Configuring resource limits to prevent runaway computations
  - Protecting against module imports that could access system resources
  - Ensuring graceful failure for problematic parameter sets

### Practical Applications

The techniques demonstrated in this tutorial are particularly valuable for:

- **Scientific Research**: Building modular, reusable calculation libraries
- **Data Science Workflows**: Creating robust parameter exploration functions
- **AI Agent Integration**: Providing specialized domain knowledge to LLMs
- **Educational Tools**: Developing interactive scientific demonstrations

### Next Steps

To further explore Unity Catalog AI's capabilities:

1. **Explore Integration Packages**: Try connecting your functions to AI frameworks like LangChain, LlamaIndex, or directly to models from OpenAI, Anthropic, or Gemini
2. **Build Domain-Specific Libraries**: Create your own specialized function collections for your field
3. **Implement Function Versioning**: Use the catalog and schema structure to maintain different versions of your functions
4. **Explore Production Deployment**: Move from local testing to serverless production deployment

By leveraging these powerful development workflow enhancements, you can create more maintainable, secure, and effective scientific functions that seamlessly integrate with modern AI systems.